<img src="http://www.exalumnos.usm.cl/wp-content/uploads/2015/06/Isotipo-Negro.gif" title="Title text" width="20%" height="20%" />


<hr style="height:2px;border:none"/>
<h1 align='center'> INF-393/578 Máquinas de Aprendizaje - 2019-1 </h1>

<H3 align='center'> Tarea 2  </H3>
<hr style="height:2px;border:none"/>

**Temas**  
* Manipulaciones básicas en numpy
* Preprocesamiento y exploración de datos, creación de datos sinteticos
* Regresión Lineal Regularizada: _Ridge_ y _Lasso_
* SVM y Kernels
* Más _Cross Validation_


**Formalidades**  
* Equipos de trabajo de 2 personas (*Ambos estudiantes deben estar preparados para presentar la tarea el día de la entrega*)
* El entregable debe ser un _Jupyter Notebook_ incluyendo los códigos utilizados, los resultados, los gráficos realizados y comentarios. Debe seguir una estructura similar a un informe (se debe introducir los problemas a trabajar, presentar los resultados y discutirlos). Si lo prefiere puede entregar un _Jupyter Notebook_ por pregunta o uno por toda la tarea, con tal de que todos los entregables esten bien identificados y se encuentren en el mismo repositorio de _Github_.
* Se debe preparar una presentación del trabajo realizado y sus hallazgos. El presentador será elegido aleatoriamente y deberá apoyarse en el _Jupyter Notebook_ que entregarán. 
* Formato de entrega: envı́o de link del repositorio en _Github_ (en caso de ser repositorio privado, invitar como colaborador al usuario de github "avalderr") al correo electrónico del ayudante (*<alvaro.valderrama.13@sansano.usm.cl>*), en copia al profesor (*<cvalle@inf.utfsm.cl>*). Especificar el siguiente asunto: [INF393/578-2019 Tarea2]
* Fecha de entrega y presentaciones: 31 de Mayo. Hora límite de entrega: 23:00. Cualquier _commit_ luego de la hora límite no será evaluado. Se realizará descuento por atrasos en envío del mail. 

<hr style="height:2px;border:none"/>

La tarea se divide en dos partes:

[1.](#primero) Clasificación de datos no-linealmente separables  
[2.](#tercero) Regularización para regresión   

La tarea tiene ejemplos de códigos con los cuales pueden guiarse en gran parte, sin embargo solo son guias y pueden ser creativos al momento de resolver la tarea. Soluciones creativas o elegantes serán valoradas. También en algunas ocaciones se hacen elecciones arbitrarias, ustedes pueden realizar otras elecciones con tal de que haya una pequeña justificación de por qué su elección es mejor o equivalente.
Recuerden intercalar su código con comentarios y con celdas _Markdown_ con los comentarios de la pregunta y con cualquier analisis, fórmula (en $ \LaTeX $) o explicación que les parezca relevante para justificar sus procedimientos. 
Noten que en general cuando se les pide elegir algo o proponer algo no se evaluará mucho la elección en si, en cambio la argumentación detrás de la elección será lo más ponderado.
Si algun modelo se demora demasiado en correr en su maquina, no olvide que puede correr _Jupyter Notebooks_ en _Collab_ de Google, esto puede ser relevante para las maquinas más lentas al momento de realizar exploraciones con _K-folds_ por ejemplo.

<a id="primero"></a>
## 1. Clasificación de datos no-linealmente separables

Algunos modelos de clasificación buscan implicita o explicitamente separar los datos linealmente, es decir ajustar un hiperplano que separe en 2 subespacios el espacio de las variables, separando ambas clases del _target_. Sin embargo en la mayoría de los casos reales esto resulta imposible. Esto no significa que no podamos ajustar modelos con estos datos o buscar transformaciones que nos permitan utilizarlos. 
    
![Alt Text](http://people.cs.uchicago.edu/~dinoj/manifold/swissroll.gif) 

[comment]: <> (Swiss Roll)

En esta primera parte de la tarea crearemos manualmente un set de datos categóricos no linealmente separables. Este se conformará de puntos en $\mathbb{R}^2$, pertenecientes a una de dos categorías etiquetadas 0 o 1, los cuales se encuentran "anidados". Probaremos distintas aproximaciones lineales y no lineales y compararemos sus desempeños.


#### 1.a Creando los datos
La función `do_circles` retorna un conjunto de puntos etiquetados, donde la etiqueta 0 corresponde a puntos siguiendo una circunferencia (con un ruido asociado) y la etiqueta 1 a puntos dentro de esa circunferencia.

Los parametros de esta función tienen nombres intencionalmente no descriptivos. Analise la función y comente interpretación puede darle a cada uno de los parámetros con respecto a la influencia que tendrán en el set de datos obtenidos. Si lo estima conveniente, cambie el nombre de los parámetros a nombres más descriptivos.

In [0]:
def do_circles(n_dots = 3000, ns = 0.15, scr = 0.3, cf = 1):
    generator = check_random_state(42)
    
    linspace = np.linspace(0, 2*cf*np.pi, n_dots)
    circle_x = np.cos(linspace)
    circle_y = np.sin(linspace)

    inner_circle_x, outer_circle_x, inner_circle_y, outer_circle_y = train_test_split(circle_x, circle_y, test_size = 0.5)
    inner_circle_x , inner_circle_y = inner_circle_x*scr , inner_circle_y*scr 

    X = np.vstack((
        np.append(outer_circle_x, inner_circle_x),
        np.append(outer_circle_y, inner_circle_y)
    )).T

    y = np.hstack([
        np.zeros(n_dots // 2, dtype=np.intp),
        np.ones(n_dots // 2, dtype=np.intp)
    ])

    X += generator.normal(scale = ns, size = X.shape)
    return train_test_split(X, y, test_size=0.3, random_state=42)

#### 1.b Graficar los datos
Cree una función que grafíque los datos, coloreando distintamente los puntos de cada clase. 
Utilice esta función para verificar gráficamente lo que propuso en la pregunta anterior.

In [0]:
def plot_circles( . . . . ):
    # . . . 
    fig1 = fig.add_subplot(121)
    fig1.scatter(X_train[y_train==0][:,0], X_train[y_train==0][:,1])
    # . . . 
    fig1.set_title("Train Set")
    # . . .

#### 1.c Regresión Logística

Realice una regresión logistica para a lo menos 5000 puntos generados con los siguientes parámetros:
* `do_circles(n_dots, 0.2, 0.3, 1)`
* `do_circles(n_dots, 0, 0.3, 1)`

Grafique los datos que está ajustando y calcule el _score_ del regresor logístico en cada caso.
¿Qué valor entrega el _score_ del regresor logístico? ¿Le parece un buen desempeño para la tarea a realizar?

In [0]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)
# . . .

#### 1.d Graficos
Para entender mejor lo que está ocurriendo con el clasificador en cada uno de los casos, grafícaremos los datos clasificados y la frontera de desición ajustada por el modelo. Para esto complete la función siguiente y luego utilicela para ambos modelos y conjuntos de datos ajustados en la pregunta anterior. 

¿Qué puede concluir luego de estas visualizaciones? ¿Es posible mejorar el desempeño de algun clasificador lineal sin hacer una transformación de los datos? ¿Qué ocurre con el clasificador si disminuimos el ruido, mejora realmente su desempeño? 

In [0]:
def plot_classifier(classifier, X_train, Y_train, X_test, Y_test, train_or_test='test'):
    plt.figure(figsize=(12, 8))
    axis = plt.axes()
    if tr_tst == 'train':
        # Scatter train points (axis.scatter(. . .)) 
    elif tr_tst == 'test':
        # Scatter test points
    XX, YY = np.mgrid[-2:2:200j, -2:2:200j] # grid for cassifier
    Z = clf.predict(np.c_[XX.ravel(), YY.ravel()]) # what the classifier predicts over the space
    Z = Z.reshape(XX.shape) # for consistency
    Zplot = Z > 0.5 # to separate both classes
    axis.pcolormesh(XX, YY, Zplot, cmap='YlGn')
    axis.contour(XX, YY, Zplot, alpha=1, colors=["r"], linestyles=["-"], levels=[0.5])
    plt.show()

Para las siguientes preguntas nos enfocaremos **solo en la versión con ruido de los datos** pues en la práctica no existen datos sin ruido. Probaremos algunos métodos no lineales para ver que tanto logramos mejorar el desempeño en nuestros datos. 

#### 1.e SVM

Entrene un clasificador SVM con los datos con ruido. ¿Qué forma tiene la frontera de desición esta vez? 

¿Mejoran los resultados al alejarse de los metodos puramente lineales? Comente sobre el desempeño del modelo y su sensibilidad a los parámetros `C` y `gamma`, explorando como cambia la frontera de desición al variar estos parámetros. 

In [0]:
from sklearn.svm import SVC
SV = SVC(C=1, gamma=5)

#### 1.f KNN
Pruebe ahora con otro método no linal, _K nearest neighbors_ o KNN. Discuta sus resultados y la frontera de desición que encuentra el modelo. 

Realice un _Cross Validation_ sobre el parámetro `n_neighbors` o `p` (o ambos usando un mallado) y comente si mejoran significativamente los resultados. Puede apoyarse de los códigos de la tarea 1 para realizar el _Cross Validation_.
¿Qué representan estos parámetros para el metodo?

In [0]:
from sklearn.neighbors import KNeighborsClassifier
KNN = KNeighborsClassifier(n_neighbors=5, p=2)

#### 1.g
Del mejor modelo obtenido en el punto anterior, grafíque los datos de test coloreando de manera difente los puntos mal clasificados por su modelo. ¿Por qué el modelo no logra clasificar bien estos puntos? ¿Qué habría ocurrido si estuvieramos utilizando los datos sin ruido?

In [0]:
# do it yourself c:

#### 1.h Metodo Kernel
Utilizaremos Kernel Gaussiano para preprocesar los datos, pasando implícitamente por un espacio altamente dimensional (más especificamente por un _manifold_ de un espacio infinito dimensional), obteniendo una representación lo más linealmente separable posible. 

Ajuste el kernel y transforme los datos. Grafique los puntos transformados y comente que realiza el modelo. Varie luego el valor de `gamma` para tratar de entender qué influencia tiene sobre la separabilidad de los puntos transformados. ¿Le parece un buen método para este problema? 

Usando algún valor de `gamma` que le parezca adecuado, ajuste un modelo de regresión logística sobre los datos transformados. Mejora el desempeño del modelo respecto a la versión original? ¿Se logra aumentar la separabilidad lineal de los datos efectivamente? Apoyese de gráficos y métricas adecuados. 

In [0]:
from sklearn.decomposition import KernelPCA
ker = KernelPCA(2,kernel='rbf',gamma=5)
ker.fit(X_train)
X_ker_train = ker.transform(X_train)
# . . 

#### 1.i Manualmente
Como nosotros conocemos como se generaron los datos tenemos una ventaja sobre los modelos. Genere una transformación de los datos que estime conveniente para obtener datos linealmente separables. Puede aumentar o disminuir la dimensión a gusto, por ejemplo puede realizar algo del estilo $(x,y)\longrightarrow (x+y)$, ó $(x,y)\longrightarrow (x,y,x+y,x-y)$. Apoyese de las carácteristicas geométricas del conjunto de datos y justifique sus elecciónes con gráficos adecuados. 

Luego entrene un regresor logístico sobre sus datos transformados y comente sobre la utilidad de conocer cómo se generaron los datos. Grafíque los datos transformados o alguna proyección de ellos junto con la frontera de desición y comente sobre lo que realiza su transformación.

<a id="tercero"></a>
## 2. Regularización para regresión. 

Un problema que podemos encontrar transversalmente a todos los modelos es el problema del _overfitting_. Este problema consiste en un modelo que aprende demasiado bien los datos de ejemplo o entrenamiento, al punto que al momento de ver ejemplos desconocidos o de validación, baja su desempeño respecto a un modelo "menos entrenado". En algunos casos podriamos decir que el modelo "aprende de memoria" los ejemplos, en vez de intentar obtener información generalizable a partir de su entrenamiento. Para el caso de la regresión, una de las aproximaciones más comunes para tratar de mitigar este efecto es el de la regularización. Agregando una penalización a la norma de los parametros de la regresión podemos restringir la eficiencia del modelo al momento de aprender los datos de entrenamiento, incluso forzando dependiendo de la intensidad de la regularización que algunos parametros se vuelvan cero. 

<img src="https://pngimage.net/wp-content/uploads/2018/06/house-for-sale-sign-png-6.png"  style="height:13.9cm;"  />

En esta pregunta utilizaremos nuevamente la base de datos de precios de casas en King County, https://www.kaggle.com/harlfoxem/housesalesprediction, que utilizamos en la Tarea 1. Utilizaremos estos datos ya conocidos y analizados anteriormente para enfocarnos en la regularización para regresión. Exploraremos como varian los distintos errores de predicción y como cambian los valores de los parámetros ajustados por el modelo para distintos valores del coeficiente de regularización, para tratar de entender que ocurre con las distintas regularizaciones y sus diferencias. 

#### 2.a Carga y preprocesamiento.
Cargue y preprocese los datos de la misma forma en que lo realizó en la tarea 1, incluyendo estandarizar el _target_. Puede reciclar sus códigos y agregar modificaciones que le parezcan pertinentes.  Para esta parte utilize 0.5 de los datos como _training set_, 0.3 como _validation set_ y 0.2 como _test set_.

#### 2.b Modelo sin regularización
Entrene un modelo de regresión lineal no regularizado con los datos preprocesados. Calcule el error cuadrático medio (MSE) del modelo para el conjunto de validación y el de entrenamiento. 

In [0]:
regressor = LinearRegression()
regressor.fit(# . . . )
    
mse = ((reg.predict(x_val)-y_val)**2).mean()

#### 2.c Regularización
Discuta que busca realizar (en términos del ajuste de los parámetros del modelo de regresión lineal) la regularización de _Lasso_. ¿Por qué se habla de penalización en norma $L^1$? Apóyese de ecuaciones. ¿Qué importancia tiene el parámetro de regularización $\lambda$? 

¿Qué diferencia hay entre _Lasso_ y _Ridge_? 

#### 2.d Influencia sobre el error
Grafique como varia el error de entrenamiento y el error de validación a medida se cambia el valor del parámetro $\lambda$. Para esto debe entrenar un modelo para cada uno de los parámetros que considerará en el gráfico y calcular el MSE para ambos sets. Comente sobre lo que ve en el gráfico y si lo encuentra pertinente puede calcular en más detalle alguna zona que le parezca interesante. 
Es conveniente entrenar primero unos cuantos modelos con valores de $\lambda$ escogidos manualmente para saber mejor en que intervalo resulta pertinente calcular estos valores (para _Ridge_ se necesitan valores más altos en este caso para ver la influencia sobre el error).

Note que el parámetro $\lambda$ es sensible a la escala de los datos, por lo cual deberá estudiar valores distintos dependiendo de si estandarizó el _target_ o no. 

In [0]:
# for example:
MSE_list = []
space = pd.np.logspace(-2,1,100)
for lambd in space:
    model = Lasso(alpha=lambd)
    # fit, error...
# . . . 
# plt.plot( space, MSE_list, . . . ) 

#### 2.e Influencia sobre los coeficientes
Grafique ahora como se comporta el valor de los coeficientes del modelo lineal en el modelo entrenado a medida crece $\lambda$, tanto para _Lasso_ como para _Ridge_. Para esto entrene un modelo para cada valor de $\lambda$ y guarde sus coeficientes, como propone el princio de código a continuación. Debe explorar valores de $\lambda$ donde todos los coeficientes sean no nulos hasta valores donde todos los coeficientes se anulan. 

¿Qué diferencias ve entre _Ridge_ y _Lasso_? ¿Coincide este comportamiento con lo que esperaría teóricamente de cada uno de los modelos? 

¿Podemos utilizar estos gráficos para discutir sobre la cantidad de información que entrega cada uno de los atributos para predecir el _target_? ¿Cual de ambos (_Ridge_ o _Lasso_) le parece mejor para este tipo de discución? Compare lo que muestran estos gráficos con alguna otra aproximación para medir la significancia de cada atributo (como correlación o _mutual information_ por ejemplo) y discuta. 

In [0]:
space = pd.np.linspace(0.0001,3,100) 
# should be fine for Lasso, for Ridge try space = pd.np.logspace(0,10,50) and plot x on a logarithmic
# scale (you can simply do np.linspace(0,10,50) when setting x values for the plot)
coefs = []
for l in space:
    model = # . . .
    
    # . . .
    
    coefs.append(model.coef_)

for y_arr, label in zip(pd.np.squeeze(coefs).T, x_cols):
    plt.plot(space, y_arr, label=label)
    # . . .

#### 2.f K-_fold_ sobre $\lambda$
Realice 5-_fold_ o 10-_fold_ sobre el parámetro de regularización $\lambda$, ya sea para _Lasso_ o _Ridge_. Explore a lo menos 20 valores distintos de $\lambda$ en un intervalo relevante luego de lo aprendido en las preguntas anteriores. Utilice MSE como el error a minimizar. 

Compare el error de entrenamiento y de validación para este modelo respecto al modelo lineal sin regularización entrenado inicialmente. ¿En su opinión se alcanza a apreciar el fenomeno de _overfitting_ para este conjunto de datos y el modelo logístico? 

#### 2.g _Grid search_ para SVM

Para terminar, comparemos el desempeño de los métodos lineales con un método no lineal, las SVM. Para esto buscaremos ajustar los hiperparámetros más relevantes del modelo, en este caso el parámetro `C` y `epsilon`, la penalización y el margen de holgura. Comente sobre la significancia de estos parámetros y su influencia sobre la máquina. Luego realice una busqueda en un mallado de estos parámetros, evaluando el _score_ (coeficiente de determinación) o MSE o ambos.

*Nota*: Entrenar una SVM toma mucho más tiempo que las regresiones lineales, por lo cual es recomendable probar primero su código en un número reducido de puntos y luego entrenar todos los modelos asegurandose de guardar bien los resultados. Si su maquina es muy lenta es incluso recomendable que esta parte la realice en _Collab_ de Google si desea explorar un número relevante de puntos. El mallado propuesto en el código es de referencia y puede elegirlo o construirlo como usted estime más conveniente. 

De todos los modelos entrenados, ¿Cuál es el mejor? ¿Cómo se compara el desempeño de este modelo con el mejor modelo lineal encontrado por usted? Adjunte gráficos que le parezcan relevantes. 

In [0]:
from sklearn.svm import SVR

Cs, epsilons = pd.np.meshgrid([i for i in range(1,10)],[i/20 for i in range(1,21)])
for for C, epsilon in zip(Cs.reshape((-1,1)), epsilons.reshape((-1,1))):
    sv = SVR(C=C, epsilon=epsilon)
    
    # fit and save the scores for each one